# ... what on such booring day to do?

In [1]:
! echo "Ähmm, what a day?"; date

Ähmm, what a day?
Sun Jul 26 00:17:24 UTC 2020


ok!, & what to do? - Looking for freebeds in state of thuringia!


so we need some hammer and steel .. hammer:

In [2]:
! echo "hammer? -ah, a hammer:";  
! curl -s https://webinstall.dev/jq | bash

hammer? -ah, a hammer:

Thanks for using webi to install 'jq' on 'linux/amd64'.
Have a problem? Experience a bug? Please let us know:
        https://github.com/webinstall/packages/issues

'jq' v1.6 already installed at /home/jovyan/.local/bin/jq


In [3]:
! echo "Is the hammer ready now?"; jq --help

Is the hammer ready now?
jq - commandline JSON processor [version 1.6]

Usage:	jq [options] <jq filter> [file...]
	jq [options] --args <jq filter> [strings...]
	jq [options] --jsonargs <jq filter> [JSON_TEXTS...]

jq is a tool for processing JSON inputs, applying the given filter to
its JSON text inputs and producing the filter's results as JSON on
standard output.

The simplest filter is ., which copies jq's input to its output
unmodified (except for formatting, but note that IEEE754 is used
for number representation internally, with all that that implies).

For more advanced filters see the jq(1) manpage ("man jq")
and/or https://stedolan.github.io/jq

Example:

	$ echo '{"foo": 0}' | jq .
	{
		"foo": 0
	}

Some of the options include:
  -c               compact instead of pretty-printed output;
  -n               use `null` as the single input value;
  -e               set the exit status code based on the output;
  -s               read (slurp) all inputs into an array; apply filte

this lightwight hammer seems as a good working tool, ok.

to hammer on it, we need also steel.

"Morgenpost" say's had some steel for us:

In [4]:
! curl curl https://interaktiv.morgenpost.de/corona-deutschland-intensiv-betten-monitor-krankenhaus-auslastung/data/intensivregister.json | tail

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0curl: (6) Could not resolve host: curl
100   365    0   365    0     0    363      0 --:--:--  0:00:01 --:--:--   735
<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">
<html><head>
<title>301 Moved Permanently</title>
</head><body>
<h1>Moved Permanently</h1>
<p>The document has moved <a href="https://interaktiv.morgenpost.de/data/corona/intensivregister.json">here</a>.</p>
<hr>
<address>Apache/2.2.22 (Debian) Server at interaktiv.morgenpost.de Port 443</address>
</body></html>


In [5]:
! echo "So go fw 2 this:"; curl https://interaktiv.morgenpost.de/data/corona/intensivregister.json | tail

So go fw 2 this:
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1165k  100 1165k    0     0   822k      0  0:00:01  0:00:01 --:--:--  822k
				null
			],
			"behandlungsschwerpunktL3": [
				null
			],
			"label": "Hufeland Klinikum, Mühlhausen",
			"faelleCovidAktuell": 2
		}
	]
}

we are lucky!

In [6]:
! curl https://interaktiv.morgenpost.de/data/corona/intensivregister.json -o BettenAllDE.json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1165k  100 1165k    0     0   901k      0  0:00:01  0:00:01 --:--:--  901k


In [25]:
! cat BettenAllDE.json | jq '.data[] | select(.krankenhausStandort.bundesland == "THUERINGEN") | .krankenhausStandort.plz + " === " + ( .faelleCovidAktuell | tostring ) + " "  '

"04600 === 3 "
"07318 === 0 "
"07548 === 3 "
"07607 === 2 "
"07747 === 3 "
"07907 === null "
"07973 === 8 "
"36433 === 1 "
"36448 === null "
"36448 === 0 "
"37308 === null "
"37339 === null "
"37355 === null "
"95693 === 0 "
"96515 === 0 "
"98527 === 4 "
"98574 === 0 "
"98617 === 1 "
"98646 === 0 "
"99089 === 7 "
"99097 === 2 "
"99310 === 2 "
"99425 === 1 "
"99437 === 5 "
"99444 === 0 "
"99510 === 0 "
"99610 === null "
"99706 === 0 "
"99734 === null "
"99752 === null "
"99768 === 5 "
"99817 === 3 "
"99867 === 0 "
"99894 === null "
"99947 === 0 "
"99955 === 0 "
"99974 === 2 "


is some steel in the "Morgenpost" in the morning?

In [8]:
! tail BettenThur.txt

"99706 2020-07-25T05:45:32Z co= 0 KMG Klinikum Sondershausen   HighCare: "
"99734 2020-07-25T06:16:58Z co= null Südharz Klinikum Nordhausen gGmbH   HighCare: VERFUEGBAR"
"99752 2020-07-25T07:01:21Z co= null Helios Klinik Bleicherode, Bleicherode   HighCare: "
"99768 2020-07-25T06:54:11Z co= 5 Lungenklinik Neustadt   HighCare: VERFUEGBAR"
"99817 2020-07-25T06:44:07Z co= 3 St. Georg Klinikum Eisenach   HighCare: VERFUEGBAR"
"99867 2020-07-25T04:59:11Z co= 0 Helios Klinikum Gotha   HighCare: "
"99894 2020-07-25T06:56:19Z co= null SRH Krankenhaus Waltershausen-Friedrichroda GmbH, Standort Friedrichroda   HighCare: VERFUEGBAR"
"99947 2020-07-25T05:03:40Z co= 0 Hufeland Klinikum, Bad Langensalza   HighCare: VERFUEGBAR"
"99955 2020-07-25T05:14:02Z co= 0 MEDIAN Klinik Bad Tennstedt, Hauptstandort   HighCare: "
"99974 2020-07-25T05:06:10Z co= 2 Hufeland Klinikum, Mühlhausen   HighCare: VERFUEGBAR"


***

Dear people in Neustadt, Eisenach and Mühlhausen: 

### We wich you best healthy!

***

In [9]:
! touch BITHLi && sort BettenThur.txt | nl | tee -a BITHLi | wc -l >> BITHLi && echo `date` >> BITHLi ; cat BITHLi

     1	"04600 2020-07-25T06:44:38Z co= 3 Klinikum Altenburger Land   HighCare: VERFUEGBAR"
     2	"07318 2020-07-25T07:51:41Z co= 0 Georgius Agricola   HighCare: VERFUEGBAR"
     3	"07548 2020-07-25T04:26:43Z co= 3 SRH Wald-Klinikum Gera   HighCare: VERFUEGBAR"
     4	"07607 2020-07-25T06:47:08Z co= 2 Waldkliniken Eisenberg   HighCare: VERFUEGBAR"
     5	"07747 2020-07-25T07:14:32Z co= 3 Universitätsklinikum Jena Lobeda   HighCare: VERFUEGBAR"
     6	"07907 2020-07-25T04:49:43Z co= null Kreiskrankenhaus Schleiz GmbH   HighCare: VERFUEGBAR"
     7	"07973 2020-07-25T06:58:33Z co= 8 Kreiskrankenhaus Greiz   HighCare: VERFUEGBAR"
     8	"36433 2020-07-25T05:18:57Z co= 1 Klinikum Bad Salzungen   HighCare: VERFUEGBAR"
     9	"36448 2020-07-24T06:21:42Z co= 0 m&i-Fachklinik Bad Liebenstein   HighCare: NICHT_VERFUEGBAR"
    10	"36448 2020-07-25T06:23:23Z co= null MEDIAN Heinrich-Mann-Klinik, MEDIAN Heinrich-Mann-Klinik Bad Liebenstein   HighCare: "
    11	"37308 2020-07-25T07:32:20Z co= null E

we will not count the people WITH covid, 

only we SHOW as example working with the tool

Our best wishes to the people in: 

Altenburg, Gera, Jena, Eisenberg, Greiz, Bad Salzungen, Suhl, Meiningen, Arnstadt, Bad Berka, Weimar, Neustadt, Eisenach und Mühlhausen! 

## Be quick healthy again!

Every suggestion, mostly for code, are very welcome. 

## Keep healthy!

In [10]:
! date ; echo "this was all for the 1st ... CU! :)  "

Sun Jul 26 00:17:36 UTC 2020
this was all for the 1st ... CU! :)  
